In [1]:
from dotenv import load_dotenv
import pandas as pd
import csv
import os
import base64
import requests
import json
from pprint import pprint
from ids import id
from ids import secret

#For Genius API

import numpy as np
import lyricsgenius

In [2]:
load_dotenv()
client_id = id
client_secret = secret

# Asegurarse de que los Id's están presentes, porque si no, la función get token da error.
# print(client_id, client_secret)

In [3]:
# Definiendo funciones <3

def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization" : "Basic " + auth_base64,
        "Content-Type" : "application/x-www-form-urlencoded"
    }

    data = {"grant_type" : "client_credentials"}

    result = requests.post(url, headers = headers, data = data)
    json_response = json.loads(result.content)
    token = json_response["access_token"]
    return token

def get_auth_header(token):
    return{"Authorization" : "Bearer " + token}

def search_for_playlist(token, playlist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={playlist_name}&type=playlist&limit=1"
    query_url = url + query
    result = requests.get(query_url, headers = headers)
    json_result_playlist = json.loads(result.content)["playlists"]["items"]
    if len(json_result_playlist) == 0:
        print("No playlist with this name found...")
        return None
    
    # pprint(json_result_playlist)
    return json_result_playlist[0]

def get_songs_in_playlist(token, artist_id):
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}"
    headers = get_auth_header(token)
    result = requests.get(url, headers = headers)
    json_result = json.loads(result.content)["tracks"]["items"]
    # pprint(json_result)
    return json_result


In [4]:
token = get_token()
result = search_for_playlist(token, "Top Canciones 2019")
playlist_id = result["id"]
songs = get_songs_in_playlist(token, playlist_id)
song_list = []
for idx, song in enumerate(songs):
    # print(f'{idx + 1}. {song["track"]["name"]}')
    song_data = {"Song":song["track"]["name"],
                 "Artist":song["track"]["artists"][0]["name"],
                 "Song Lenght (ms)":song["track"]["duration_ms"],
                 "Popularity":song["track"]["popularity"],
                 "Release date":song["track"]["album"]["release_date"],
                 "Song Id":song["track"]["id"]}
    song_list.append(song_data)
# pprint(song_list)
# pprint(songs)
spoti_19 = pd.DataFrame(song_list)
spoti_19.head()

,Song,Artist,Song Lenght (ms),Popularity,Release date,Song Id
0,Calma - Remix,Pedro Capó,238200,77,2018-10-05,5iwz1NiezX7WWjnCgY5TH4
1,Con Calma,Daddy Yankee,193226,83,2019-01-24,5w9c2J52mkdntKOmRLeM2m
2,Te Vi,Piso 21,231848,80,2018-12-14,059bcIhyc2SBwm6sw2AZzd
3,Otro Trago,Sech,225933,75,2019-04-19,1Ej96GIBCTvgH7tNX1r3qr
4,Callaita,Bad Bunny,250533,80,2019-05-31,2TH65lNHgvLxCKXM3apjxI


In [5]:
#Tomar la columna Song Id del DataFrame con los Top Tracks y convertir los valores en una lista de strings separadas por comas, para pasarlo así al request de los Audio Features

id_list = spoti_19["Song Id"]
id_list = ",".join(id_list)
id_list


'5iwz1NiezX7WWjnCgY5TH4,5w9c2J52mkdntKOmRLeM2m,059bcIhyc2SBwm6sw2AZzd,1Ej96GIBCTvgH7tNX1r3qr,2TH65lNHgvLxCKXM3apjxI,0s5DNDV6LAKn2u2Ufa3VjQ,1LIqNVhaIgxXUt2eVoIQJ3,5W83ErFkO3aKAIS1WMi6u0,0fea68AdmYNygeTGI4RC18,6FyRXC8tJUh863JCkyWqtk,7KbF6AdprOXEEHlsq11Z6d,1c0hsvHLELX6y8qymnpLKL,2qG5sZ7Si6sdK74qLxedYM,46lvmzK8wxAy66tjzXXSh0,2ksOAxtIxY8yElEWw8RhgK,2beS6dD5rovBxSA1zYoSHR,7g8YaUQABMal0zWe7a2ijz,3f5lA8nnc4YYF1qf78WpHw,5BDP2tky8oMQJPS33frKVp,3gE4eQH3K83Sght0ZLvuBK,6WkJ2OK163XXS2oARUC9JM,3KkXRkHbMCARz0aVfEt68P,6ocbgoVGwYJhOv1GgI9NsF,0TK2YIli7K1leLovkQiNik,2Fxmhks0bxGSBdJ92vM42m,6o8ogTrSX8H0F66Pdz3Tsc,7hbk4BaF66O09R1gq0bKwj,4lzxJ4jCuFDXXGkE1LmpKR,5PXsZaT8cOxb4mXrA20PMQ,4PJ3umtW1aWdfh5EITHc6T,0KHhNlHN5lWn7q332PiPys,0KoiMHhqKLoL4xM54rcWYY,25ZAibhr3bdlMCLmubZDVt,0akyEssGRVHstqCSWXusJL,4THusowxdHxwt0HBh5pyRx,32lexhcz93AinKK0qLTZaZ,1LiN0Z98FkR1t0m8KmLcAH,1nocRtwyNPVtGcIJqfgdzZ,7BuHdm4lQqcyG3qTwdkxus,4Clmg1g8n2yBYG13Z1oSdG,4yVO0wOhXsZC1RuPgHP524,2LIh4uzqq9cXMPzzmcToHl,1mT47pKLXUwPYGwB91GcNz,6De0lHrwBf

In [6]:
#Request para los Audio Features

def get_AF_in_songs(token, id_list):
    url = f"https://api.spotify.com/v1/audio-features?ids={id_list}"
    headers = get_auth_header(token)
    response = requests.get(url, headers = headers)
    response_json = response.json()
    #pprint(response_json)
    return response_json

In [7]:
token = get_token()
audio_features_data = get_AF_in_songs(token, id_list)["audio_features"]
audio_features_list = []


for idx, af in enumerate(audio_features_data):
    audio_features = {"Acousticness":af["acousticness"],
                 "Danceability":af["danceability"],
                 "Energy":af["energy"],
                 "Instrumentalness":af["energy"],
                 "Liveness":af["energy"],
                 "Speechiness":af["speechiness"],
                 "Tempo":af["tempo"],
                 "Valence":af["valence"]}
    audio_features_list.append(audio_features)
audio_features_df = pd.DataFrame(audio_features_list)
#audio_features_df

spoti_19_af_df = pd.concat([spoti_19, audio_features_df], axis = 1)
spoti_19_af_df

,Song,Artist,Song Lenght (ms),Popularity,Release date,Song Id,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Speechiness,Tempo,Valence
0,Calma - Remix,Pedro Capó,238200,77,2018-10-05,5iwz1NiezX7WWjnCgY5TH4,0.3230,0.826,0.773,0.773,0.773,0.0524,126.899,0.761
1,Con Calma,Daddy Yankee,193226,83,2019-01-24,5w9c2J52mkdntKOmRLeM2m,0.1100,0.737,0.860,0.860,0.860,0.0593,93.989,0.656
2,Te Vi,Piso 21,231848,80,2018-12-14,059bcIhyc2SBwm6sw2AZzd,0.0446,0.877,0.777,0.777,0.777,0.1170,102.020,0.706
3,Otro Trago,Sech,225933,75,2019-04-19,1Ej96GIBCTvgH7tNX1r3qr,0.1360,0.746,0.700,0.700,0.700,0.3410,176.044,0.619
4,Callaita,Bad Bunny,250533,80,2019-05-31,2TH65lNHgvLxCKXM3apjxI,0.6000,0.610,0.624,0.624,0.624,0.3090,176.169,0.244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Sucker,Jonas Brothers,181026,84,2019-06-07,22vgEDb5hykfaTwLuskFGD,0.0427,0.842,0.734,0.734,0.734,0.0588,137.958,0.952
96,Cuando Nadie Ve,Morat,219586,82,2018-10-25,71iAechwVKGu26pahzFL0k,0.1330,0.806,0.778,0.778,0.778,0.0616,95.993,0.840
97,Que Le De,Rauw Alejandro,225600,60,2019-01-25,08aYFNUTIOMGq93e2VSArQ,0.1900,0.770,0.750,0.750,0.750,0.0785,99.997,0.609
98,Ponle,Rvssian,164764,67,2018-09-06,28O1p0bQXH2F6Qn8jBZZR4,0.1880,0.857,0.772,0.772,0.772,0.1950,94.030,0.589


In [8]:
client_access_token="KKVSlBN3kbpuuRtePodUd-RTz3CVxexH7EOatwekQVhbjLwWAHNSsiT9Mbf2vYMi"
LyricsGenius = lyricsgenius.Genius(client_access_token)

lyrics_df = spoti_19_af_df.Song
lyrics_df = pd.DataFrame(lyrics_df)
lyrics_df.head(10)



,Song
0,Calma - Remix
1,Con Calma
2,Te Vi
3,Otro Trago
4,Callaita
5,Desconocidos
6,HP
7,Secreto
8,LA CANCIÓN
9,Adan y Eva


In [9]:
# The package got some timeout issue so these two lines are needed. If you don't then there will be error when you scrape
# Source: https://github.com/johnwmillr/LyricsGenius/issues/121
LyricsGenius.timeout = 15  #timeout
LyricsGenius.sleep = 5

In [10]:
#Quick test first song

sample_song = lyrics_df.Song[0]
print(f"Name of the first song in the database is: {sample_song}")

# Search for the song's lyric
searched_song = LyricsGenius.search_song(sample_song)
print(f"The lyric is:\n{searched_song.lyrics}")

Name of the first song in the database is: Calma - Remix
Searching for "Calma - Remix"...
Done.
The lyric is:
24 ContributorsCalma (Remix) Lyrics[Letra de "Calma (Remix)"]

[Intro: Pedro Capó & Farruko]
Laramercy gang
Welcome to the paradise
¡Farru!
Cuatro abrazos y un café
Apenas me desperté
Y al mirarte recordé
Que ya todo lo encontré
Tu mano en mi mano
De todo escapamos
Juntos ver el sol caer
(Let them enjoy the paradise)

[Coro: Pedro Capó]
Vamos pa' la playa, pa’ curarte el alma
Cierra la pantalla, abre la Medalla
Todo el mar Caribe, viendo tu cintura
Tú le coqueteas, tú eres busca bullas y me gusta

[Post-Coro: Pedro Capó]
Lento y contento, cara al vie-ento
Lento y contento, cara al vie-ento

[Verso 1: Farruko]
Ya, ya, ya, ¡Farru!
Y aprovecha que el sol 'tá caliente y vamo' a disfrutar el ambiente
Vámono’ a meternos pa'l agua pa' que veas qué rico se siente
Y vámono' en tropical, por to'a la costa a chinchorrear
De chinchorro a chinchorro paramo' a darno' una Medalla
Bien fría, p

In [11]:
# Create an array to store each song's lyric
lyrics_arr = []

# Traverse through the database, get the song's lyrics from title, and do some preprocessing
for i in range(len(lyrics_df)):
    # get title
    song_title = lyrics_df.Song[i]
    
    # search for song in genius.com
    searched_song = LyricsGenius.search_song(song_title)
    
    # if we can't find a song's lyrics then skip and append empty string
    if searched_song is None:
        lyrics_arr.append("")
        continue
        
    # get the lyric
    lyric = searched_song.lyrics
    
    # replace the lyrics newline with ". "
    lyric = lyric.replace("\n", ". ")
    
    # remove initial non-lyrics character:
    # Source: https://thispointer.com/remove-string-before-a-specific-character-in-python/
    # lyric = lyric[lyric.index('.') + 1 :]
    
    # append the processed lyric to the array
    lyrics_arr.append(lyric)

Searching for "Calma - Remix"...
Done.
Searching for "Con Calma"...
Done.
Searching for "Te Vi"...
Done.
Searching for "Otro Trago"...
Done.
Searching for "Callaita"...
Done.
Searching for "Desconocidos"...
Done.
Searching for "HP"...
Done.
Searching for "Secreto"...
Done.
Searching for "LA CANCIÓN"...
Done.
Searching for "Adan y Eva"...
Done.
Searching for "11 PM"...
Done.
Searching for "Soltera (Remix)"...
Done.
Searching for "Con Altura"...
Done.
Searching for "Tal Vez"...
Done.
Searching for "China"...
Done.
Searching for "No Me Conoce - Remix"...
Done.
Searching for "Pa Mí - Remix"...
Done.
Searching for "Ella Quiere Beber - Remix"...
Done.
Searching for "Un Año"...
Done.
Searching for "MIA"...
Done.
Searching for "Dancin (feat. Luvli) - Krono Remix"...
Done.
Searching for "Sunflower - Spider-Man: Into the Spider-Verse"...
Done.
Searching for "7 rings"...
Done.
Searching for "Señorita"...
Done.
Searching for "bad guy"...
Done.
Searching for "Baby Girl (feat. Lalo Ebratt)"...
Done.

In [12]:
# Check length
len(lyrics_arr)

100

In [13]:
# Append lyric columns
lyrics_df['lyrics'] = lyrics_arr

# Replace "" with Nan
#lyrics_df = lyrics_df.replace('', np.nan)

# Drop Nan
#lyrics_df = lyrics_df.dropna(subset = ["lyrics"])

lyrics_df

,Song,lyrics
0,Calma - Remix,"24 ContributorsCalma (Remix) Lyrics[Letra de ""..."
1,Con Calma,31 ContributorsTranslationsEnglishCon Calma Ly...
2,Te Vi,"30 ContributorsToda (Remix) Lyrics[Letra de ""T..."
3,Otro Trago,"11 ContributorsOtro Trago Lyrics[Letra de ""Otr..."
4,Callaita,44 ContributorsTranslationsEnglishDeutschItali...
...,...,...
95,Sucker,166 ContributorsTranslationsDeutschไทย / Phasa...
96,Cuando Nadie Ve,11 ContributorsCuando Nadie Ve Lyrics[Letra de...
97,Que Le De,77 ContributorsTranslationsDeutschGasolina Lyr...
98,Ponle,"7 ContributorsPonle Lyrics[Letra de ""Ponle""]. ..."


# Bag of words

In [14]:
# Create a bag-of-words column
lyrics_df['bag'] = lyrics_df.lyrics.map(lambda t: t.replace('!','').replace('.','').lower().split())
lyrics_df.head()

,Song,lyrics,bag
0,Calma - Remix,"24 ContributorsCalma (Remix) Lyrics[Letra de ""...","[24, contributorscalma, (remix), lyrics[letra,..."
1,Con Calma,31 ContributorsTranslationsEnglishCon Calma Ly...,"[31, contributorstranslationsenglishcon, calma..."
2,Te Vi,"30 ContributorsToda (Remix) Lyrics[Letra de ""T...","[30, contributorstoda, (remix), lyrics[letra, ..."
3,Otro Trago,"11 ContributorsOtro Trago Lyrics[Letra de ""Otr...","[11, contributorsotro, trago, lyrics[letra, de..."
4,Callaita,44 ContributorsTranslationsEnglishDeutschItali...,"[44, contributorstranslationsenglishdeutschita..."


In [15]:
# Create a number of words in each bag of words
lyrics_df['len'] = lyrics_df.bag.map(len)
lyrics_df.head()

,Song,lyrics,bag,len
0,Calma - Remix,"24 ContributorsCalma (Remix) Lyrics[Letra de ""...","[24, contributorscalma, (remix), lyrics[letra,...",478
1,Con Calma,31 ContributorsTranslationsEnglishCon Calma Ly...,"[31, contributorstranslationsenglishcon, calma...",628
2,Te Vi,"30 ContributorsToda (Remix) Lyrics[Letra de ""T...","[30, contributorstoda, (remix), lyrics[letra, ...",945
3,Otro Trago,"11 ContributorsOtro Trago Lyrics[Letra de ""Otr...","[11, contributorsotro, trago, lyrics[letra, de...",649
4,Callaita,44 ContributorsTranslationsEnglishDeutschItali...,"[44, contributorstranslationsenglishdeutschita...",503


In [16]:
# Create a group of bag of words in case we happen to have 2 similar titles
g = lyrics_df.groupby('Song')
g.size()

Song
11 PM                1
7 rings              1
A Través del Vaso    1
Adan y Eva           1
Adictiva             1
                    ..
Verte Ir             1
Ya No Tiene Novio    1
Yo x Ti, Tu x Mi     1
bad guy              1
thank u, next        1
Length: 100, dtype: int64

In [17]:
title_bags = g.bag.aggregate(lambda bag: [w for b in bag.values for w in b])
title_bags

Song
11 PM                [8, contributors11, pm, lyrics[letra, de, "11,...
7 rings              [473, contributorstranslationstürkçeespañolrom...
A Través del Vaso    [6, contributorsa, través, del, vaso, lyrics[l...
Adan y Eva           [18, contributorsadán, y, eva, lyrics[letra, d...
Adictiva             [17, contributorsadictiva, lyrics[letra, de, "...
                                           ...                        
Verte Ir             [17, contributorsverte, ir, lyrics[letra, de, ...
Ya No Tiene Novio    [15, contributorsya, no, tiene, novio, lyrics[...
Yo x Ti, Tu x Mi     [48, contributorstranslationsportuguêsespañoly...
bad guy              [339, contributorstranslationshrvatskitürkçesv...
thank u, next        [398, contributorstranslationstürkçesvenskaesp...
Name: bag, Length: 100, dtype: object

In [18]:
# Compute Term Frequency Matrix
TF_o = title_bags.apply(lambda bag: pd.Series(bag).value_counts())
#trasponer
TF = TF_o.T
#Drop pronombres
TF=TF.drop(["sus","tu","esta","ti","eso"],axis=0)
#Reduccion de posesivos
TF=TF.drop(["su","mí","tus","tu'","mis"],axis=0)
#drop expresiones
TF=TF.drop(["(¡eh)","(¡shu)","(¡jajá)","ey)","(tss,","eh","(ey)","(yeh-eh)","yeh-eh-eh","(woh-oh;","oh","oh;","¡shiuf","o-o-ovy","(yeh-eh-eh)9embed","(mmm,","[estribillo:"],axis=0)
#drop artículos
TF=TF.drop(["los"],axis=0)
#drop nonjunciones
TF=TF.drop(["y","ni"],axis=0)
#errores
TF=TF.drop(["(eh-eh)","uah,","na'","oah,","oah","uah","si","eh,","ah,","[intro]","e’","la’","like[verso","(-la,","-la)","lo'","yeh-eh-eh)","(yeh-eh-eh)","18","ah…","(ah)","[interludio]","[puente]","[estribillo]","(¡wuh)","ah","oh,","¿la","[verso","la'","(woh)","[pre-estribillo]","esto'","(no)","-la,","(woh-oh,","e'","do'","yeh-eh)","ey","(yeh)","se","es","6","¿a","(eh,","11","e","yep)469embed","contributorstranslationstürkçeespañolromânăportuguêspolskiitalianohebrewελληνικάdeutschfrançaisnederlandsdanskفارسی7","473","(oh,","yeh)","yeh,","(yeh,","está","no-no)","eh)","ba-ba-baby4embed","(uh)","(uh-ah)","[verso]","contributors11","(eh-eh,","lyrics[letra","yeah","8","(no-no)","no-oh)","[outro]","ba-ba-ba-,","me","lo","(no-no,","e","no,","[pre-coro]","[coro]","una","él)","(pa'","(no","oh-oh-oh","pa'","¿cómo","¿que","él","el","de","a","en","la","tú","que","te","no","un", "o", "u", "pero", "aunque","contributorstranslationstürkçesvenskaespañolрусскийportuguêspolski한국어deutschfrançaisnederlands​thank","so","yee512embed","in","at","on","tú"],axis=0)

TF

Song,11 PM,7 rings,A Través del Vaso,Adan y Eva,Adictiva,Adiós Amor,Aerolínea Carrillo,Ahora Te Puedes Marchar,Amanece,Amigos Con Derechos,...,Te Vi,Tra Tra Tra Remix - Remix,Tutu,Un Año,Vaina Loca,Verte Ir,Ya No Tiene Novio,"Yo x Ti, Tu x Mi",bad guy,"thank u, next"
mejor?,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ves,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
por,9.0,NaN,3.0,NaN,1.0,NaN,2.0,4.0,NaN,NaN,...,8.0,1.0,1.0,3.0,9.0,6.0,1.0,79.0,NaN,NaN
seas,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
merece,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
havin',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
met,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
worried,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
"so,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [19]:
#Get all the column names (each word)
low=TF_o.columns.values.tolist()
print(type(low))


<class 'list'>


In [20]:
#DATAFRAME OUT OF LIST OF WORDS
low_df=pd.DataFrame(data=low,columns=['Words'])
low_df


,Words
0,que
1,te
2,no
3,tú
4,mejor?
...,...
8991,contributorstranslationstürkçesvenskaespañolру...
8992,worried
8993,"so,"
8994,"now,"


In [21]:
#remove all the non alphatical chats
low_df['Words']=low_df['Words'].str.replace(r'\W',"")
low_df

C:\Users\Beto Mtz\AppData\Local\Temp\ipykernel_20940\1769907625.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  low_df['Words']=low_df['Words'].str.replace(r'\W',"")


,Words
0,que
1,te
2,no
3,tú
4,mejor
...,...
8991,contributorstranslationstürkçesvenskaespañolру...
8992,worried
8993,so
8994,now


In [22]:
#Getting how many times words are repeated trough the series
freq=low_df.groupby('Words').size()
#words=low_df["Words"].unique()
#getting dataframe
wf_df=pd.DataFrame({"Frequency":freq})
wf_df


,Frequency
Words,
,9
0000,1
011119,1
042,1
051019,1
...,...
último,2
única,2
único,1


In [23]:
#CSV FREQUENCY
gecsv=freq.to_frame()
gecsv.to_csv("FREQUENCY88.csv")


In [24]:
#clean dataframe with counts above 3
df3=wf_df.loc[wf_df["Frequency"]>3]
df3.to_csv("clean_freq.csv")
df3

,Frequency
Words,
,9
1,4
2,4
3,4
5,4
...,...
yehyehyehyeh,4
yo,12
you,7
